In [1]:
#set adequate environment
import os
import sys
import theano
import keras
theano.config.gcc.cxxflags = '-march=corei7'

Using Theano backend.


In [2]:
#load needed things
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, Adagrad, Adadelta, RMSprop
from keras.layers import Input, Activation, Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import backend as K
# Run classifier with cross-validation and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
import itertools
import math
#import ROOT
import cPickle as pickle

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
#load events
comparison = '>='
njets = 2
min4lmass = 118
max4lmass = 130

filein = open('hzz4l_vbf_selection_noDjet_m4l118-130GeV_shuffledFS.pkl','r')
events = pickle.load( filein )
filein.close()

In [4]:
print '----- Available MCs -----'
print events.keys()
print ''
print '----- Available Variables -----'
print events[events.keys()[0]].keys()

----- Available MCs -----
['ttZ', 'ZH', 'VBF', 'qqZZ', 'WH', 'ttH', 'ggZZ', 'Data', 'HJJ']

----- Available Variables -----
['theta2', 'j3e', 'l1phi', 'l3sip', 'njets', 'j4phi', 'j1e', 'j2pt', 'l1charge', 'l4pt', 'theta1', 'l2pdg', 'isomax', 'final_state', 'l1pdg', 'j6phi', 'j1eta', 'mc_sumweight', 'l1pt', 'DjetVAJHU', 'l4eta', 'j2e', 'z2mass', 'pfmet', 'l1eta', 'j5e', 'genmet', 'phistar1', 'j3eta', 'l3pdg', 'l4sip', 'l3pfx', 'deltajj', 'j4pt', 'l2sip', 'costheta1', 'costheta2', 'Djet', 'j2eta', 'phi', 'l1sip', 'eta4l', 'mass4l', 'l2eta', 'j6pt', 'j5eta', 'l3charge', 'l2pt', 'l2pfx', 'j2phi', 'mT', 'nbjets', 'dphi', 'z1mass', 'thetastar', 'costhetastar', 'l4charge', 'l3pt', 'l2charge', 'event_weight', 'pt4l', 'j5pt', 'mc', 'j1phi', 'j1pt', 'j4eta', 'sipmax', 'j6eta', 'l4pdg', 'l4pfx', 'massjj', 'l4phi', 'j3pt', 'l2phi', 'l3phi', 'l3eta', 'j4e', 'j3phi', 'l1pfx', 'j5phi', 'j6e', 'event_index']


In [5]:
#shows events and statistics
for ik in events:
    nevents = len(events[ik]['mc'])
    sumw = events[ik]['mc_sumweight'][0]
    print '%s events: %i, normalized: %.4f' % (ik,nevents,sumw)
    
#only MC
summ = 0
for ik in events:
    if(ik != 'Data'):
        summ += len(events[ik]['mc'])
print 'MC total events (absolute): %i' % summ

ttZ events: 90, normalized: 0.1357
ZH events: 9152, normalized: 0.4467
VBF events: 38243, normalized: 3.3486
qqZZ events: 210, normalized: 1.4985
WH events: 21492, normalized: 0.6767
ttH events: 5201, normalized: 0.1541
ggZZ events: 854, normalized: 0.1468
Data events: 16, normalized: 16.0000
HJJ events: 7168, normalized: 7.2095
MC total events (absolute): 82410


In [ ]:
fig = pyp.figure()
fig.set_figheight(6)
fig.set_figwidth(14)
pyp.rc("font", size=18)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
fig1 = fig.add_subplot(121)
composition = []
components = []
for ik in events:
    if(ik != 'Data'):
        composition.append( len(events[ik]['mc']) )
        components.append(ik)
pyp.pie(composition, labels=components, autopct='%1.1f%%', shadow=True, startangle=0)
pyp.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
pyp.title('Njets {0} {1}, m4l = [{2}, {3}]GeV'.format(comparison,njets,min4lmass,max4lmass))

fig2 = fig.add_subplot(122)
masses4l = []
weights = []
dmasses4l = []
nsig = 0
nbkg = 0
for ik in events:
    if(ik != 'Data'):
        w = []
        m = []
        for iev in range(len(events[ik]['mc'])):
            w.append(events[ik]['event_weight'][iev])
            m.append(events[ik]['mass4l'][iev])
            if(ik == 'VBF'):
                nsig += events[ik]['event_weight'][iev]
            else:
                nbkg += events[ik]['event_weight'][iev]
        weights.append(w)
        masses4l.append(m)
    else:
        for iev in range(len(events[ik]['mc'])):
                dmasses4l.append(events[ik]['mass4l'][iev])
                

bins = np.linspace(115,135,10)
pyp.hist(masses4l, bins, weights=weights, histtype='bar', stacked=True, label=components)

n, bins, _ = pyp.hist(dmasses4l, bins, histtype='step', color='white')
mid = 0.5*(bins[1:] + bins[:-1])
eyh = []
eyl = []
teyh = 0
teyl = 0
nm4l = 0
for i in range(len(n)):
    if(n[i] != 0):
        nm4l += n[i]
        eyl.append( -0.5+math.sqrt(n[i]+0.25) )
        teyl += (-0.5+math.sqrt(n[i]+0.25))*(-0.5+math.sqrt(n[i]+0.25))
        eyh.append( 0.5+math.sqrt(n[i]+0.25) )
        teyh += (0.5+math.sqrt(n[i]+0.25))*(0.5+math.sqrt(n[i]+0.25))
    else:
        eyl.append(0)
        eyh.append(0)
    #print 'eyl: %.2f, eyh: %.2f' % (eyl[i],eyh[i])
m4l_err = [eyl,eyh]
pyp.errorbar(mid, n, yerr=m4l_err, fmt='o', color='black', label='Data')

pyp.xlabel('m4l')
pyp.ylabel('Normalized')
#pyp.ylim([0.001,10])
pyp.legend(loc='upper right')
#pyp.gca().set_yscale("log")


print "Observed = %.1f +%.1f -%.1f" % (nm4l,math.sqrt(teyh),math.sqrt(teyl))
print "Signal = %.1f, Background = %.1f, S+B = %.1f" % (nsig,nbkg,nsig+nbkg)

pyp.tight_layout()
fig = pyp.show()

In [ ]:
from UserFunctions import *
#organize events and splits mc into train, validation and test; data it's kept full
#the split_factor sets train size then validation and test sets have the same size
split_factor = 0.8
full_event_train, full_event_test, full_event_data = splitInputs(events, split_factor)

In [ ]:
#states the variables to be used as input for the network
#spectator variables (as Djet, DjetVAJHU, weights..) are automatically imported
use_vars = [
            'l1pt','l1eta','l1phi'
            ,'l2pt','l2eta','l2phi'
            ,'l3pt','l3eta','l3phi'
            ,'l4pt','l4eta','l4phi'
            ,'j1pt','j1eta','j1phi'
            ,'j2pt','j2eta','j2phi'
            ,'j3pt','j3eta','j3phi'
            #,'njets'
            #,'pfmet'
           ]

In [ ]:
#prepare train set
X = {}
Y = {}
X['train'], Y['train'], Ydjet_train, Ymela_train, weights_train_set, scales_train_set = prepareSet(full_event_train, use_vars)

In [ ]:
#prepare test set
X['test'], Y['test'], Ydjet_test, Ymela_test, weights_test_set, scales_test_set = prepareSet(full_event_test, use_vars)

In [ ]:
#before standardization #cross checking of inputs
print X['train'][0]

#from sklearn import preprocessing
#normalize inputs to [0, 1]
#X['train'] = preprocessing.normalize(X['train'])
#X['test'] = preprocessing.normalize(X['test'])
#print X['train'][0]

#standardize to have mean = 0 and sigma = 1
#X['train'] = preprocessing.scale(X['train'])
#X['test'] = preprocessing.scale(X['test'])
#print X['train'][0]

#plot input variables
#fig = pyp.figure()
#fig.set_figheight(14)
#fig.set_figwidth(14)
#for ivar in range(len(use_vars)):
#    svar = []
#    sweight = []
#    bvar = []
#    bweight = []
#    for iev in range(len(Y['test'])):
#        if(Y['test'][iev] == 1):
#            sweight.append( weights_train_set[iev] )
#            svar.append( X['test'][iev][ivar] )
#        else:
#            bweight.append( weights_train_set[iev] )
#            bvar.append( X['test'][iev][ivar] )

#    fig.add_subplot(5,5,ivar+1)
#    xmin = min(bvar)
#    xmax = max(bvar)
#    if(min(svar) < min(bvar)):
#        xmin = min(svar)
#    if(max(svar) > max(bvar)):
#        xmax = max(svar)
#    nbins = int((xmax - xmin)/0.01)
#    bins = np.linspace(xmin, xmax, nbins)
#    pyp.hist(svar, bins, weights=sweight, alpha=0.5, color='red', label='sig')
#    pyp.hist(bvar, bins, weights=bweight, alpha=0.5, color='blue', label='bkg')

#pyp.tight_layout()
#fig = pyp.show()    

In [ ]:
# updatable plot
from IPython.display import clear_output
class Tscheduler(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.accs = []
        self.val_accs = []
        self.logs = []
        self.trocs = []
        self.vrocs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.accs.append(logs.get('acc'))
        self.val_accs.append(logs.get('val_acc'))
        Y_score = dnn_model.predict(X['train'])
        fpr, tpr, thresholds = roc_curve(Y['train'], Y_score, sample_weight=weights_train_set)
        roc_auc = auc(fpr, tpr, reorder=True)
        self.trocs.append(roc_auc)
        Y_score = dnn_model.predict(X['test'])
        fpr, tpr, thresholds = roc_curve(Y['test'], Y_score, sample_weight=weights_test_set)
        roc_auc = auc(fpr, tpr, reorder=True)
        self.vrocs.append(roc_auc)
        self.i += 1
        
        clear_output(wait=True)
        fig = pyp.figure()
        fig.set_figheight(6)
        fig.set_figwidth(13)
        
        fig1 = fig.add_subplot(121)
        pyp.plot(self.x, self.losses, label="tloss: %f" % self.losses[len(self.losses)-1]) 
        pyp.plot(self.x, self.val_losses, label="vloss: %f" % self.val_losses[len(self.val_losses)-1])
        #pyp.plot(self.x, self.accs, label="tacc: %f" % self.accs[len(self.accs)-1]) 
        #pyp.plot(self.x, self.val_accs, label="vacc: %f" % self.val_accs[len(self.val_accs)-1])
        #c_lrate = K.eval(self.model.optimizer.lr)
        #pyp.title("Current LR: %f" % c_lrate)
        #c_momentum = K.eval(self.model.optimizer.momentum)
        #pyp.title("Current M: %f" % c_momentum)
        pyp.xlabel('epochs')
        pyp.ylabel('Loss')
        pyp.legend()
        #pyp.ylim([0.7,1.2])
        pyp.grid(True)
        #pyp.show()
        
        fig2 = fig.add_subplot(122)
        pyp.plot(self.x, self.trocs, color='blue', label='Current train AUC = %0.3f' % self.trocs[len(self.trocs)-1])
        pyp.plot(self.x, self.vrocs, color='red', label='Current   test AUC = %0.3f' % self.vrocs[len(self.vrocs)-1])
        #pyp.ylim([0, 1.0])
        pyp.xlabel('epochs')
        pyp.ylabel('AUC')
        pyp.title('ROC evolution')
        pyp.legend()
        pyp.grid(True)
        #pyp.show()

        pyp.tight_layout()
        fig = pyp.show()

        #changing lr based on loss
        #if(len(self.losses) > 20):
        #    lsum1 = 0
        #    lsum2 = 0
        #    for i in range(6):
        #        if(i < 3):
        #            lsum1 += self.losses[len(self.losses)-1-i]/3.
        #        else:
        #            lsum2 += self.losses[len(self.losses)-1-i]/3.
        #    if(math.fabs(lsum2-lsum1) < 0.01):
        #        n_lrate = c_lrate*(1 - 1/20.)
        #        K.set_value(self.model.optimizer.lr, n_lrate)
        #else:
        #    n_lrate = self.losses[len(self.losses)-1]*0.001
        #    K.set_value(self.model.optimizer.lr, n_lrate)
        
        #initial_lrate = 0.001
        #drop = 0.6
        #epochs_drop = 100
        #lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        #K.set_value(self.model.optimizer.lr, 0.001)
        
        #stop training if training and testing Loss get too away
        #if(epoch > 50):
        #    train_sum = 0
        #    val_sum = 0
        #    for i in range(30):
        #        train_sum += self.losses[len(self.losses)-1-i]/6.
        #        val_sum += self.val_losses[len(self.val_losses)-1-i]/6.
        #    if(train_sum-val_sum < -0.02):
        #        self.model.stop_training = True
        #        print "Stoping training!"
                
            
Tschedule = Tscheduler()

In [ ]:
#parameters for training
nepochs = 10
wait_for = 60
sbatch = 86
opt = Adam()
early_stopping = EarlyStopping(monitor='val_loss', patience=wait_for)

In [ ]:
#DNN network
#from keras.layers import Dropout
ninputs = len(X['train'][0])
print "---------- DNN Topology --------- "
print "Inputs: %i" % ninputs

dnn_model = Sequential()
dnn_model.add(Dense(7, input_shape=(ninputs,), activation='relu', kernel_initializer='random_uniform'))
dnn_model.add(Dense(7, activation='relu', kernel_initializer='random_uniform'))
dnn_model.add(Dense(7, activation='relu', kernel_initializer='random_uniform'))    
dnn_model.add(Dense(1, activation='sigmoid', kernel_initializer='random_uniform'))    
dnn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print dnn_model.summary()

In [ ]:
filepath1="best_weights.hdf5"
checkpoint = ModelCheckpoint(filepath1, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

#train the DNN
history1 = dnn_model.fit(X['train'],
                         Y['train'], 
                         sample_weight=scales_train_set,
                         #sample_weight=weights_train_set,
                         validation_data=(X['test'],
                                          Y['test']
                                          ,scales_test_set
                                          #,weights_test_set
                                         ), 
                         epochs=nepochs,
                         batch_size=sbatch, 
                         verbose=0,
                         callbacks=[checkpoint, early_stopping,Tschedule]
                       )
print 'Intputs: ', use_vars

In [ ]:
# load final best weights
dnn_model.load_weights(filepath1)

In [ ]:
fig = pyp.figure(figsize=(10,10))

#train set normalized roc
Y_score = dnn_model.predict(X['train'])
fpr, tpr, thresholds = roc_curve(Y['train'], Y_score, sample_weight=weights_train_set)
roc_auc = auc(fpr, tpr,reorder=True)
pyp.plot(fpr, tpr, linestyle='--', color='blue', label='DNN train AUC = %0.3f' % roc_auc)

#test set unormalized roc
Y_score = dnn_model.predict(X['test'])
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr,reorder=True)
pyp.plot(fpr, tpr, color='red', label='DNN test AUC = %0.3f' % roc_auc)

#random
pyp.plot([0, 1], [0, 1], linestyle='--', lw=2, color='black', label='Luck')

pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
fig = pyp.figure(figsize=(10,10))

#normalized rocs
fpr, tpr, thresholds = roc_curve(Y['test'], Ydjet_test, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr,reorder=True)
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='blue', label=('Djet test AUC = %0.3f' % roc_auc))

fpr, tpr, thresholds = roc_curve(Y['test'], Ymela_test, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr,reorder=True)
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='red', label=('MELA test AUC = %0.3f' % roc_auc))

Y_score = dnn_model.predict(X['test'])
fpr, tpr, thresholds = roc_curve(Y['test'], Y_score, sample_weight=weights_test_set)
roc_auc = auc(fpr, tpr,reorder=True)
pyp.plot(fpr, tpr, linestyle='--', lw=2, color='green', label=('DNN test AUC = %0.3f' % roc_auc))


pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()

In [ ]:
#Make some plots to compare the discriminants
signal_sumw = 0
for iev in range(len(full_event_test['VBF']['mc'])):
    signal_sumw += full_event_test['VBF']['event_weight'][iev]

djet_sb = []
djet_seff = []
djet_ep = []
mela_sb = []
mela_seff = []
mela_ep = []
dnn_sb = []
dnn_seff = []
dnn_ep = []
djet_cuts = []
mela_cuts = []
dnn_cuts = []

efull_sdjet = []
efull_bdjet = []
efull_smela = []
efull_bmela = []
efull_sdnn = []
efull_bdnn = []
esweights = []
ebweights = []

for cut in range(200):
    icut = cut/100.
    
    s_djet = 0
    s_mela = 0
    s_dnn = 0
    b_djet = 0
    b_mela = 0
    b_dnn = 0
    for i in range(len(Y['test'])):
        djet = Ydjet_test[i]
        mela = Ymela_test[i]
        dnn = Y_score[i][0]
        weight = weights_test_set[i]
        
        #VBF (signal)
        if(Y['test'][i] == 1):
            if(cut == 0):
                efull_sdjet.append( djet )
                efull_smela.append( mela )
                efull_sdnn.append( dnn )
                esweights.append( weight )
            
            if(djet > icut):
                s_djet += weight
            if(mela > icut):
                s_mela += weight
            if(dnn > icut):
                s_dnn += weight
        
        #Backgrounds
        else:
            if(cut == 0):
                efull_bdjet.append( djet )
                efull_bmela.append( mela )
                efull_bdnn.append( dnn )
                ebweights.append( weight )
            
            if(djet > icut):
                b_djet += weight
            if(mela > icut):
                b_mela += weight
            if(dnn > icut):
                b_dnn += weight
    
    if(s_djet != 0 or b_djet != 0):
        djet_ep.append( (s_djet/signal_sumw)*(s_djet/(s_djet+b_djet)) )

    if(s_mela != 0 or b_mela != 0):
        mela_ep.append( (s_mela/signal_sumw)*(s_mela/(s_mela+b_mela)) )
        
    if(s_dnn != 0 or b_dnn != 0):
        dnn_ep.append( (s_dnn/signal_sumw)*(s_dnn/(s_dnn+b_dnn)) )

    if(b_djet > 0 or s_djet > 0):
        djet_sb.append(s_djet/math.sqrt(b_djet+s_djet))
        djet_seff.append(s_djet/signal_sumw)
        djet_cuts.append(icut)
    
    if(b_mela > 0 or s_mela > 0):
        mela_sb.append(s_mela/math.sqrt(b_mela+s_mela))
        mela_seff.append(s_mela/signal_sumw)
        mela_cuts.append(icut)
    
    if(b_dnn > 0 or s_dnn > 0):
        dnn_sb.append(s_dnn/math.sqrt(b_dnn+s_dnn))
        dnn_seff.append(s_dnn/signal_sumw)
        dnn_cuts.append(icut)

In [ ]:
#compute the same quantities above for training set
tfull_sdjet = []
tfull_bdjet = []
tfull_smela = []
tfull_bmela = []
tfull_sdnn = []
tfull_bdnn = []
tsweights = []
tbweights = []

TrainScore = dnn_model.predict(X['train'])
for iev in range(len(Y['train'])):
    djet = Ydjet_train[iev]
    mela = Ymela_train[iev]
    dnn = TrainScore[iev][0]
    weight = weights_train_set[iev]

    if(Y['train'][iev] == 1):
        tfull_sdjet.append( djet )
        tfull_smela.append( mela )
        tfull_sdnn.append( dnn )
        tsweights.append( weight )
    else:
        tfull_bdjet.append( djet )
        tfull_bmela.append( mela )
        tfull_bdnn.append( dnn )
        tbweights.append( weight )

In [ ]:
index = np.argmax(djet_ep)
print 'Djet: Max Seff*Purity = %.3f, S/sqrt(S+B) = %.3f, cut = %.3f' % (djet_ep[index], djet_sb[index], djet_cuts[index])
index = np.argmax(mela_ep)
print 'MELA: Max Seff*Purity = %.3f, S/sqrt(S+B) = %.3f, cut = %.3f' % (mela_ep[index], mela_sb[index], mela_cuts[index])
index = np.argmax(dnn_ep)
print 'DNN: Max Seff*Purity = %.3f, S/sqrt(S+B) = %.3f, cut = %.3f' % (dnn_ep[index], dnn_sb[index], dnn_cuts[index])

#current metric we adopted
fig = pyp.figure()
fig.set_figheight(7)
fig.set_figwidth(14)

fig1 = fig.add_subplot(121)
pyp.plot(djet_seff, djet_ep, lw=1, color='blue', label='Djet: %.3f'% auc(djet_seff, djet_ep,reorder=True))
pyp.plot(mela_seff, mela_ep, lw=1, color='red', label='MELA: %.3f'% auc(mela_seff, mela_ep,reorder=True))
pyp.plot(dnn_seff, dnn_ep, lw=1, color='green', label='DNN: %.3f'% auc(dnn_seff, dnn_ep,reorder=True))
pyp.xlim([0, 1])
pyp.xlabel('Signal eff')
pyp.ylim([0, 1])
pyp.ylabel('eff*purity')
pyp.legend()
pyp.grid(True)

fig2 = fig.add_subplot(122)
pyp.plot(djet_seff, djet_sb, lw=1, color='blue', label='Djet: %.3f'% auc(djet_seff, djet_sb,reorder=True))
pyp.plot(mela_seff, mela_sb, lw=1, color='red', label='MELA: %.3f'% auc(mela_seff, mela_sb,reorder=True))
pyp.plot(dnn_seff, dnn_sb, lw=1, color='green', label='DNN: %.3f'% auc(dnn_seff, dnn_sb,reorder=True))
pyp.xlim([0, 1])
pyp.xlabel('Signal eff')
#pyp.ylim([0, 1])
pyp.ylabel('S/(S+B)')
pyp.legend()
pyp.grid(True)

pyp.tight_layout()
fig = pyp.show()

In [ ]:
fig = pyp.figure()
fig.set_figheight(5)
fig.set_figwidth(14)

fig1 = fig.add_subplot(131)
bins = np.linspace(0, 3, 50)
pyp.hist(tfull_sdjet, bins, normed=1, alpha=0.2, color='red', label='s-train')#, weights=tsweights)
pyp.hist(tfull_bdjet, bins, normed=1, alpha=0.2, color='blue',label='b-train')
pyp.hist(efull_sdjet, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='red', label='s-test')
pyp.hist(efull_bdjet, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='blue',label='b-test')
pyp.xlabel('Djet')
pyp.ylabel('Normalized')
pyp.ylim([0.002,100])
pyp.legend()
pyp.gca().set_yscale("log")

fig2 = fig.add_subplot(132)
bins = np.linspace(-0.05, 1.05, 50)
pyp.hist(tfull_smela, bins, normed=1, alpha=0.2, color='red', label='s-train')
pyp.hist(tfull_bmela, bins, normed=1, alpha=0.2, color='blue',label='b-train')
pyp.hist(efull_smela, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='red', label='s-test')
pyp.hist(efull_bmela, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='blue',label='b-test')
pyp.xlabel('MELA')
pyp.ylim([0.002,100])
pyp.legend()
pyp.gca().set_yscale("log")

fig3 = fig.add_subplot(133)
bins = np.linspace(-0.05, 1.05, 50)
pyp.hist(tfull_sdnn, bins, normed=1, alpha=0.2, color='red', label='s-train')
pyp.hist(tfull_bdnn, bins, normed=1, alpha=0.2, color='blue',label='b-train')
pyp.hist(efull_sdnn, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='red', label='s-test')
pyp.hist(efull_bdnn, bins, normed=1, linestyle='--', histtype='step', alpha=0.5, color='blue',label='b-test')
pyp.xlabel('DNN')
pyp.ylim([0.002,100])
pyp.legend()
pyp.gca().set_yscale("log")

pyp.tight_layout()
fig = pyp.show()

In [ ]:
#creates normalized ROCs separated by MC
print '******** ROCs from DNN *********'
fig = pyp.figure(figsize=(10,10))
colors = ['red','blue','violet','green','orange','yellow','cyan']

Y_truth = {}
Y_mela = {}
X = {}
Weights = {}
for ik in full_event_test:
    Y_truth[ik] = []
    X[ik] = []
    Weights[ik] = []
    Y_mela[ik] = []

    for iev in range(len(full_event_test[ik]['mc'])):
        variables = []
        for ivar in use_vars:
            variables.append(full_event_test[ik][ivar][iev])
        X[ik].append(variables)
        if(ik == 'VBF'):
            Y_truth[ik].append(1)
        else:
            Y_truth[ik].append(0)
        Weights[ik].append(full_event_test[ik]['event_weight'][iev])
                
ic = 0
for ik in Y_truth:
    if(ik == 'VBF'):
        continue
    cX = np.concatenate([X['VBF'],X[ik]])
    cY = np.concatenate([Y_truth['VBF'],Y_truth[ik]])
    cW = np.concatenate([Weights['VBF'],Weights[ik]])
    Y_score = dnn_model.predict(cX)
    fpr, tpr, thresholds = roc_curve(cY, Y_score, sample_weight=cW)
    roc_auc = auc(fpr, tpr,reorder=True)
    print 'VBF vs ',ik,' -- ROC AUC: %.2f' % roc_auc
    pyp.plot(fpr, tpr, lw=1, color=colors[ic], label=('VBF vs %s (%0.2f)' % (ik,roc_auc)))
    ic += 1
    
#------------------------------------------------------------------------------------------------------------
print ''
print '******** ROCs from MELA *********'
for ik in full_event_test:
    for iev in range(len(full_event_test[ik]['mc'])):    
        Y_mela[ik].append(full_event_test[ik]['DjetVAJHU'][iev])

ic = 0
for ik in Y_truth:
    if(ik == 'VBF'):
        continue
    sY = np.concatenate([Y_mela['VBF'],Y_mela[ik]])
    cY = np.concatenate([Y_truth['VBF'],Y_truth[ik]])
    cW = np.concatenate([Weights['VBF'],Weights[ik]])
    fpr, tpr, thresholds = roc_curve(cY, sY, sample_weight=cW)
    roc_auc = auc(fpr, tpr,reorder=True)
    print 'VBF vs ',ik,' -- ROC AUC: %.2f' % roc_auc
    pyp.plot(fpr, tpr, linestyle='--', lw=1, color=colors[ic], label=('VBF vs %s (%0.2f)' % (ik,roc_auc)))
    ic += 1
    
pyp.xlim([0.001, 1.0])
pyp.ylim([0.001, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
#pyp.xscale('log')
#pyp.yscale('log')
pyp.title('Receiver Operating Characteristic')
pyp.legend(loc="lower right")
pyp.grid(True)
pyp.show()    